In [ ]:
import tkinter as tk
# from PIL import Image, ImageTk

window = tk.Tk()
window.title("Sudoku")
# icon = ImageTk.PhotoImage(file= 'icon.png')
# window.iconphoto(True, icon)

import tkinter as tk

canvas = tk.Canvas(window, width=750, height=750, bg="white")
canvas.grid()

case_taille = 50
sudoku_taille = 9 * case_taille

for ligne in range(10): # 9 : nb de lignes
    if ligne % 3 == 0:
        epaisseur = 3
    else:
        epaisseur = 1
    canvas.create_line(50, ligne * case_taille + 50, sudoku_taille + 50, ligne * case_taille + 50, width= epaisseur)

for col in range(10): # 9 : nb de colonne
    if col % 3 == 0:
        epaisseur = 3
    else:
        epaisseur = 1
    canvas.create_line(col * case_taille + 50, 50, col * case_taille + 50, sudoku_taille + 50, width= epaisseur)

window.mainloop()